In [46]:
import pandas as pd
import numpy
import numpy.fft
from numpy import *
import numpy as np
import pymongo
import re
from numpy  import array
import librosa
import os
import ntpath
import pathlib
from pathlib import Path
import pickle
from tinytag import TinyTag
from pydub import AudioSegment
from pydub.utils import make_chunks
from itertools import tee, islice, chain

import timbral_models
import math
from math import *
from math import atan2,degrees
from numpy.lib import stride_tricks
import scipy
from scipy.fftpack import *
from __future__ import division
import scipy.stats
import scipy.stats.mstats
import matplotlib.pyplot as plt

In [47]:
df = pd.read_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/audio_information_april_6.pkl")

In [48]:
class Spectrum(numpy.ndarray):

    def __new__(subtype, shape, dtype=float, buffer=None, offset=0,
          strides=None, order=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = numpy.ndarray.__new__(subtype, shape, dtype, buffer, offset, strides,
                         order)

        obj.sampleRate = 0

        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).

        self.sampleRate = getattr(obj, 'sampleRate', None)

        # We do not need to return anything

    #####################
    # Spectrum methods
    #####################

    def centroid(self):
        """
        Compute the spectral centroid.
        Characterizes the "center of gravity" of the spectrum.
        Approximately related to timbral "brightness"
        """
        binNumber = 0

        numerator = 0
        denominator = 0

        for bin in self:
            # Compute center frequency
            f = (self.sampleRate / 2.0) / len(self)
            f = f * binNumber

            numerator = numerator + (f * abs(bin))
            denominator = denominator + abs(bin)

            binNumber = binNumber + 1

        return (numerator * 1.0) / denominator

   

    def crest(self):
        """
        Compute the spectral crest factor, i.e. the ratio of the maximum of the spectrum to the
        sum of the spectrum
        """
        absSpectrum = abs(self)
        spectralSum = numpy.sum(absSpectrum)

        maxFrequencyIndex = numpy.argmax(absSpectrum)
        maxSpectrum = absSpectrum[maxFrequencyIndex]

        return maxSpectrum / spectralSum

    def flatness(self):
        """
        Compute the spectral flatness (ratio between geometric and arithmetic means)
        """
        geometricMean = scipy.stats.mstats.gmean(abs(self))
        arithmeticMean = numpy.sum(abs(self)) / len(self)

        return geometricMean / arithmeticMean

    def kurtosis(self):
        """
        Compute the spectral kurtosis (fourth spectral moment)
        """
        return scipy.stats.kurtosis(abs(self))

    def mean(self):
        """
        Compute the spectral mean (first spectral moment)
        """
        return numpy.sum(abs(self)) / len(self)

    def plot(self):
        """
        Plot the spectrum using matplotlib
        """
        plt.plot(abs(self))
        plt.xlim(0, len(self))
        plt.show()

   
        # Convert the index into a frequency
        frequency = rolloffIndex * (self.sampleRate / 2.0) / len(self)
        return frequency

    def skewness(self):
        """
        Compute the spectral skewness (third spectral moment)
        """
        return scipy.stats.skew(abs(self))

    def spread(self):
        """
        Compute the spectral spread (basically a variance of the spectrum around the spectral centroid)
        """
        centroid = self.centroid()

        binNumber = 0

        numerator = 0
        denominator = 0

        for bin in self:
            # Compute center frequency
            f = (self.sampleRate / 2.0) / len(self)
            f = f * binNumber

            numerator = numerator + (((f - centroid) ** 2) * abs(bin))
            denominator = denominator + abs(bin)

            binNumber = binNumber + 1

        return math.sqrt((numerator * 1.0) / denominator)

    def variance(self):
        """
        Compute the spectral variance (second spectral moment)
        """
        return numpy.var(abs(self))

In [66]:
class FeatureUtility:
    
    df_myclass = pd.DataFrame() 
    index = 0
    waveform_array = np.ndarray(0)
    sr = 0
     # parameterized constructor 
    def __init__(self, df_myclass, index): 
        self.df_myclass = df 
        self.index = index
        self.waveform_array =  pickle.loads(df['audio_waveform'][index])
        self.sr = df['sampling_rate'][index]
        
    
    def getVocalPercentage(self):
        full = self.df_myclass['full_spectrum_mean'][self.index]
        background = self.df_myclass['instruments_mean'][self.index]
        foreground = self.df_myclass['vocals_mean'][self.index]
        m =full/(background+foreground)
        full_new = full/m
        return (foreground/full_new)*100        

    def getAnglesBetweenConsecutiveBeats(self):
        beat_strength = []
        beat_times = pickle.loads(self.df_myclass['beat'][self.index])
        onset_envelope  = pickle.loads(self.df_myclass['onset_strength'][self.index])
        for j in  beat_times:
            beat_strength.append(onset_envelope[j])
        return  self.getArrayOfAnglesBetweenConsecutivepointsInArray(beat_strength)

    # Determines the angle of a straight line drawn between point one and two.
    def getAngleBetweenTwoPoints(self,p1x,p1y,p2x,p2y):
        xDiff = p2x - p1x
        yDiff = p2y - p1y
        angle = degrees(atan2(yDiff, xDiff))
        if(angle >= 0):
            return angle
        else:
            return (angle+360)
    def getPreviousAndNext(self,iterable_data):
        prevs, items, nexts = tee(iterable_data, 3)
        prevs = chain([None], prevs)
        nexts = chain(islice(nexts, 1, None), [None])
        return zip(prevs, items, nexts)
    def getArrayOfAnglesBetweenConsecutivepointsInArray(self,iterable_data):
        xx = 1
        yy = 2
        np_arr =  np.array([])
        for previous, item, nxt in self.getPreviousAndNext(iterable_data):
            if(nxt == None):
                np_arr = np.append(np_arr,0)
            else:
                tempvar = self.getAngleBetweenTwoPoints(xx,item,yy,nxt)
                np_arr = np.append(np_arr,tempvar)
        return np_arr
    def getAdditionalSpectralFeatures(self):
        windowFunction = numpy.hamming
        test_1 = pickle.loads(self.df_myclass['audio_waveform'][self.index])
        fixedFrames = frames(test_1,self.df_myclass['sampling_rate'][self.index],1024)
        spectra = [self.spectrum(f,self.df_myclass['sampling_rate'][self.index]) for f in fixedFrames]
        return self.getSpectralFeaturesDict(spectra)

    def getSpectralFeaturesDict(self,spectra):
        crest = numpy.ndarray(0)
        kurtosis = numpy.ndarray(0)                      
        mean = numpy.ndarray(0)                          
        skewness =numpy.ndarray(0)                       
        spread = numpy.ndarray(0)                         
        variance = numpy.ndarray(0)
        i = 0
        while i < (len(spectra)):
            crest = numpy.append(crest,spectra[i].crest())
            kurtosis = numpy.append(kurtosis,spectra[i].kurtosis())                       # Spectral Kurtosis
            mean = numpy.append(mean,spectra[i].mean())                           # Spectral Mean
            skewness = numpy.append(skewness,spectra[i].skewness())                       # Spectral Skewness
            spread = numpy.append(spread,spectra[i].spread())                         # Spectral Spread
            variance = numpy.append(variance,spectra[i].variance())                       # Spectral Variance
            i += 1

#         dict_spec = {}
#         dict_spec['crest'] = crest
#         dict_spec['kurtosis'] =  kurtosis                     
#         dict_spec['mean'] =        mean                   
#         dict_spec['skewness'] =    skewness                   
#         dict_spec['spread'] =     spread                     
#         dict_spec['variance'] = variance
        return [crest,kurtosis,mean,skewness,spread,variance]
    def spectrum(self,arr,sr):
        """
        Compute the spectrum using an FFT
        Returns an instance of Spectrum
        """
        fftdata = numpy.fft.rfft(arr) # rfft only returns the real half of the FFT values, which is all we need
        spectrum = fftdata.view(Spectrum)
        spectrum.sampleRate = sr

        return spectrum
    def getSpectralContrast(self):
        return self.df_myclass['spectral_contrast'][self.index]
    def getSpectralFlatness(self):
        return self.df_myclass['spectral_flatness'][self.index]
    def getSpectralBandwidth(self):
        return self.df_myclass['spectral_bandwidth'][self.index]
    def getSpectralCentroid(self):
        return self.df_myclass['spectral_centroid'][self.index]
    def getSpectralRolloff(self):
        return self.df_myclass['rolloff'][self.index]
    def getOnsetDetect(self):
        return self.df_myclass['onset_detect'][self.index]
    def getOnsetStrength(self):
        return self.df_myclass['onset_strength'][self.index]
    def getTimbralBrightness(self):
        return timbral_models.timbral_brightness(self.waveform_array, fs=self.sr)
    def getTimbralHardness(self):
        return timbral_models.timbral_hardness(self.waveform_array, fs=self.sr)
    def getTimbralDepth(self):
        return timbral_models.timbral_depth(self.waveform_array, fs=self.sr)
    def getTimbralRoughness(self):
        return timbral_models.timbral_roughness(self.waveform_array, fs=self.sr)
    def getTimbralSharpness(self):
        return timbral_models.timbral_sharpness(self.waveform_array, fs=self.sr)
    def getTimbralBooming(self):
        return timbral_models.timbral_booming(self.waveform_array, fs=self.sr)
    def getTimbralReverb(self):
        return timbral_models.timbral_reverb(self.waveform_array, fs=self.sr)
    

In [67]:
def frames(arr,sr, frameSize, windowFunction = None):
    """
    Decompose this frame into smaller frames of size frameSize
    """
    frames = []
    start = 0
    end = frameSize
    while start < len(arr):

        if windowFunction == None:
            frames.append(arr[start:end])
        else:
            window = windowFunction(frameSize)
            window.shape = (frameSize, 1)
            window =  numpy.squeeze(window)
            frame = arr[start:end]
            if len(frame) < len(window):

                diff = len(window) - len(frame)
                frame = numpy.append(frame, [0] * diff)



            windowedFrame = (frame) * window
            frames.append(windowedFrame)

        start = start + frameSize
        end = end + frameSize

    return frames

In [68]:
def getmetadata(mycol):
    displaynames_meta = []
    for x in mycol.find():
        displaynames_meta.append(x['name'])
    return displaynames_meta

def addFeatureToDataFrame(count_row,metadata):
    i = 1
    while (i<=count_row):
        df_new.loc[i] =getFeatureList(i,metadata)
        i += 1


In [69]:
def getFeatureList(i,metadata):
    featurelist = []
    for m in metadata:
        featurelist.append(df[m][i])
    ap = FeatureUtility(df,i)
    featurelist.append(ap.getAnglesBetweenConsecutiveBeats())
    featurelist.append(ap.getVocalPercentage())
    featurelist.append(ap.getSpectralContrast())
    featurelist.append(ap.getSpectralFlatness())
    featurelist.append(ap.getSpectralBandwidth())
    featurelist.append(ap.getSpectralCentroid())
    featurelist.append(ap.getSpectralRolloff())
    list_2 = ap.getAdditionalSpectralFeatures()
    featurelist = featurelist+list_2
    print("concatnated" + str(len(featurelist)))
    featurelist.append(ap.getOnsetDetect())
    featurelist.append(ap.getOnsetStrength())
    featurelist.append(ap.getTimbralBrightness())
    featurelist.append(ap.getTimbralHardness())
    featurelist.append(ap.getTimbralDepth())
    featurelist.append(ap.getTimbralRoughness())
    featurelist.append(ap.getTimbralSharpness())
    featurelist.append(ap.getTimbralBooming())
    featurelist.append(ap.getTimbralReverb())

    return featurelist

In [70]:
meta = ['filename','artist','duration','albumartist','bitrate','comment','filesize','genre','title','track','samplerate','track_total','year','beat','tempo']
other_features =  ['angles_between_consec_beats','vocal_percentage']
spectral_features = ['spectral_contrast','spectral_flatness','spectral_bandwidth','spectral_centroid','spectral_rolloff','spectral_crest','spectral_kurtosis','spectral_mean','spectral_skewness','spectral_spread','spectral_variance']
onset_features =  ['onset_detect','onset_strength']
timral_features = ['timbral_brightness','timbral_hardness','timbral_depth','timbral_roughness','timbral_sharpness','timbral_booming','timbral_reverb']
header = numpy.concatenate((meta, other_features,spectral_features,onset_features,timral_features))
print(header)
print(len(header))

['filename' 'artist' 'duration' 'albumartist' 'bitrate' 'comment'
 'filesize' 'genre' 'title' 'track' 'samplerate' 'track_total' 'year'
 'beat' 'tempo' 'angles_between_consec_beats' 'vocal_percentage'
 'spectral_contrast' 'spectral_flatness' 'spectral_bandwidth'
 'spectral_centroid' 'spectral_rolloff' 'spectral_crest'
 'spectral_kurtosis' 'spectral_mean' 'spectral_skewness' 'spectral_spread'
 'spectral_variance' 'onset_detect' 'onset_strength' 'timbral_brightness'
 'timbral_hardness' 'timbral_depth' 'timbral_roughness'
 'timbral_sharpness' 'timbral_booming' 'timbral_reverb']
37


In [71]:
df_new = generateDataFrameWithHeader(header)
addFeatureToDataFrame(df.shape[0],meta)         

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in float_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars


concatnated28


C:\ProgramData\Anaconda3\lib\site-packages\mkl_fft\_numpy_fft.py:158: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.fft(a, n, axis)


In [72]:
count_row_dfNew = df_new.shape[0]  # gives number of row count
print(count_row_dfNew)
df_new.head()

1


,filename,artist,duration,albumartist,bitrate,comment,filesize,genre,title,track,...,spectral_variance,onset_detect,onset_strength,timbral_brightness,timbral_hardness,timbral_depth,timbral_roughness,timbral_sharpness,timbral_booming,timbral_reverb
1,Bebe Rexha - Drinking About You (The Chainsmok...,Bebe Rexha,217.034914,The Chainsmokers Bebe Rexha,320.0,,8712257,Electronic,Drinking About You (The Chainsmokers Remix),None,...,"[0.0, 0.0, 5.331915531314735e-07, 7.9883966463...",b'\x80\x04\x95\xc6\x19\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...,62.331284,58.504801,57.175338,56.718811,52.734007,29.081699,1


In [58]:
df['sampling_rate'][1]

22050

In [73]:
df_new.to_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/audio_features_april_20.pkl",protocol = -1)
